In [9]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup as Soup
import pandas as pd
from datetime import datetime

@ movie.csv

1. 원본 (moive + link).csv로 movieId 수정과정 전처리 진행 (1회)
=> movies.csv

2. 2000년 이후 영화 전처리 진행 (1회)
=> 2000-01~Current Top 1000 Movies.csv

3. 2020년 이후 영화 전처리 진행 (1회)
=>2020_Movie_list.csv

4. comingSoon 전처리 ( 계속 업데이트할 것)
=> imdbComingSoon.csv


# 1~3

In [10]:
## 전처리 함수 1회만 실행

def Preprocessing() :
    global Total
    # movies.csv
    # links.csv 은 최초 원본파일
    
    movie = pd.read_csv('./병합 전 csv/movies.csv')
    links = pd.read_csv('./병합 전 csv/links.csv') 
    imdb_movie = pd.merge(movie, links, on = 'movieId')
    imdb_movie = imdb_movie[['imdbId', 'title', 'genres']]
    imdb_movie = imdb_movie.rename(columns = {'imdbId':'imdb_movie_Id'})
    
    movie2000 = pd.read_csv('./2000-01~Current Top 1000 Movies.csv')
    movie2000 = movie2000[['Movie_No','Name','Genre']]
    movie2000 = movie2000.rename(columns = {'Movie_No':'imdb_movie_Id', 'Name':'title', 'Genre':'genres'})

    movie2020 = pd.read_csv('./2020_Movie_list.csv')
    movie2020 = movie2020[['Movie_No','Name','Genre', 'Director', 'Actor_list']]
    movie2020 = movie2020.rename(columns = {'Movie_No':'imdb_movie_Id', 'Name':'title', 'Genre':'genres', 'Director':'Director_name', 'Actor_list':'Actor_name'})

    Total = pd.concat([imdb_movie, movie2000, movie2020], axis = 0)
    return Total

In [11]:
Preprocessing()

,imdb_movie_Id,title,genres,Director_name,Actor_name
0,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
1,113497,Jumanji (1995),Adventure|Children|Fantasy,NaN,NaN
2,113228,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN
3,114885,Waiting to Exhale (1995),Comedy|Drama|Romance,NaN,NaN
4,113041,Father of the Bride Part II (1995),Comedy,NaN,NaN
...,...,...,...,...,...
995,14941222,Injil,Action,Pil Gam-Sung,"['Jung-min Hwang', 'Lee Ho-jung', 'Jae-Bum Kim..."
996,10572658,Collection,Thriller,Marianna Palka,"['Alex Pettyfer', 'Shakira Barrera', 'Mike Vog..."
997,10505316,Illusions perdues,"Drama, History",Xavier Giannoli,"['Benjamin Voisin', 'Cécile de France', 'Vince..."
998,10833270,Upside-Down Magic,"Family, Fantasy",Joe Nussbaum,"['Izabela Rose', 'Siena Agudong', 'Kyle Howard..."


# 4.


In [12]:
def get_new_movies(Total) :
    Name = []
    MovieId = []
    Year = []
    Rating = []
    Meta_score = []
    Votes = []
    Genre = []
    Director = []
    Stars = []
    
    Date = datetime.today().strftime("%Y-%m")    # YYYYmmddHHMMSS 형태의 시간 출력

    
    url = get('https://www.imdb.com/movies-coming-soon/' + Date + '/?ref_=cs_dt_pv')
    request = url.text
    soup_data = Soup(request, 'html.parser')

    movies = soup_data.findAll('div', {'class' : 'list_item odd'})
    movies1 = soup_data.findAll('div', {'class' : 'list_item even'})
    for i in movies + movies1:
        # 영화 제목
        Name.append(i.h4.a.text[:-7])
        # 영화 년도
        Year.append(i.h4.a.text[-5:-1])
        # 영화 ID
        MovieId.append(i.h4.a["href"][-8:-1])
        # 영화 장르
        tmp = i.select('p> span')
        tmp = list(filter(lambda x: str(x)[:6] == '<span>', tmp))
        tmp = list(map(lambda x: str(x).replace('<span>','').replace('</span>',''), tmp))
        Genre.append(tmp)
        # 영화 감독
        Director.append(i.find('div', {'class' : 'txt-block'}).span.text.strip())
        # 영화 배우

        Stars_ = list(map(lambda x: str(x).replace('</a>',''), i.select('div.txt-block> a')))
        result = []
        for e in Stars_:
            sign = False
            a_result = ''

            for c in e :
                if sign:
                    a_result += c

                elif c == '>':
                    sign = True

            result.append(a_result)

        Stars.append(result)


        # Meta Score
        try :
            Meta_score.append(i.tr.find('div', {'class':'rating_txt'}).span.text.strip())
        except Exception as err:
            Meta_score.append(0)
    
    data = list(zip(MovieId, Name, Year, Genre, Meta_score, Director,Stars))
    df = pd.DataFrame(data, columns = ['imdb_movie_Id', 'title', 'Year', 'genres','Meta_score', 'Director_name','Actor_name'])
    df = df[['imdb_movie_Id', 'title', 'genres', 'Director_name', 'Actor_name']]
    Total = pd.concat([Total,df], axis = 0)
    Total = Total.drop_duplicates(['imdb_movie_Id'])
    Total = Total.set_index('imdb_movie_Id')
    
    return Total.to_csv('./병합 후 csv/Total.csv')

In [13]:
get_new_movies(Total)